Looking at the Hopkins COVID-19 data that was cloned from https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime as dt

# It turns out that I don't think I need the statistics library after all
# import statistics


df_confirmed = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

#df_deaths = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
#df_recovered = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [2]:
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,50,53,59,70,75,82,114,147,177,212
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,511,581,639,639,701,773,839,825,878,889
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,150,160,178,178,200,212,226,243,266,313
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,117,129,149,149,197,238,428,566,673,790


In [3]:
df_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 61 columns):
Province/State    319 non-null object
Country/Region    462 non-null object
Lat               462 non-null float64
Long              462 non-null float64
1/22/20           462 non-null int64
1/23/20           462 non-null int64
1/24/20           462 non-null int64
1/25/20           462 non-null int64
1/26/20           462 non-null int64
1/27/20           462 non-null int64
1/28/20           462 non-null int64
1/29/20           462 non-null int64
1/30/20           462 non-null int64
1/31/20           462 non-null int64
2/1/20            462 non-null int64
2/2/20            462 non-null int64
2/3/20            462 non-null int64
2/4/20            462 non-null int64
2/5/20            462 non-null int64
2/6/20            462 non-null int64
2/7/20            462 non-null int64
2/8/20            462 non-null int64
2/9/20            462 non-null int64
2/10/20           462 non-null int64
2

I think I just need to work with the confirmed DataFrame. It's set up such that each row is a specific location, and the columns are different dates.



In [4]:
#creating template for dataframes to record various values

df_template = df_confirmed.copy()


df_template.head(), df_template.size

(  Province/State Country/Region      Lat      Long  1/22/20  1/23/20  1/24/20  \
 0            NaN       Thailand  15.0000  101.0000        2        3        5   
 1            NaN          Japan  36.0000  138.0000        2        1        2   
 2            NaN      Singapore   1.2833  103.8333        0        1        3   
 3            NaN          Nepal  28.1667   84.2500        0        0        0   
 4            NaN       Malaysia   2.5000  112.5000        0        0        0   
 
    1/25/20  1/26/20  1/27/20  ...  3/9/20  3/10/20  3/11/20  3/12/20  3/13/20  \
 0        7        8        8  ...      50       53       59       70       75   
 1        2        4        4  ...     511      581      639      639      701   
 2        3        4        5  ...     150      160      178      178      200   
 3        1        1        1  ...       1        1        1        1        1   
 4        3        4        4  ...     117      129      149      149      197   
 
    3/14/20 

In [5]:
# Creating template out of JHU table format

do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

for x in df_template.columns.values:
    if (x not in do_not_include):
        for y in range(df_template['1/22/20'].values.size):
            df_template.at[y, x] = 0
            
            
df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Adding the rest of the beginning of January

for q in range(1, 22):
    day_str = str(q)
    date_str = '1/' + day_str + '/20'
    df_template[date_str] = 0    

df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_template.columns.values

array(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20',
       '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20',
       '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20',
       '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20',
       '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20',
       '3/17/20', '3/18/20', '1/1/20', '1/2/20', '1/3/20', '1/4/20',
       '1/5/20', '1/6/20', '1/7/20', '1/8/20', '1/9/20', '1/10/20',
       '1/11/20', '1/12/20', '1/13/20', '1/14/20', '1/15/20', '1/16/20',
       '1/17/20', '1/18/20', '1/19/20', '1/20/20', '1/21/20'],
      dtype=object)

In [8]:
ordered_columns = ['Province/State', 'Country/Region', 'Lat', 'Long', '1/1/20',
                  '1/2/20', '1/3/20', '1/4/20', '1/5/20', '1/6/20',
                   '1/7/20', '1/8/20', '1/9/20', '1/10/20', '1/11/20',
                   '1/12/20', '1/13/20', '1/14/20', '1/15/20', '1/16/20',
                   '1/17/20', '1/18/20', '1/19/20', '1/20/20', '1/21/20',
                   '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20',
                   '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20',
                   '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20',
                   '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20',
                   '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
                   '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20',
                   '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20',
                   '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
                   '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20',
                   '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20',
                  '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20',
                  '3/17/20', '3/18/20']

df_template = df_template[ordered_columns]

df_template.head()

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# For some reason, Greenland doesn't have a sunrise time on January 1
# I'll get this sorted out later so that it can be included in
# future datasets

df_template.rename(columns={"Country/Region": "Region"}, inplace=True)

df_template = df_template[df_template.Region != 'Greenland']

df_template.rename(columns={"Region": "Country/Region"}, inplace=True)

df_template.tail(15)

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
447,NaN,Somalia,5.1521,46.1996,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
448,NaN,Tanzania,-6.3690,34.8888,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
449,NaN,The Bahamas,24.2500,-76.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
450,Virgin Islands,US,18.3358,-64.8963,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
451,Cayman Islands,United Kingdom,19.3133,-81.2546,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452,Reunion,France,-21.1351,55.2471,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
453,NaN,Barbados,13.1939,-59.5432,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,NaN,Montenegro,42.5000,19.3000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
455,NaN,Kyrgyzstan,41.2044,74.7661,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
456,NaN,Mauritius,-20.2000,57.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_template.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461 entries, 0 to 461
Data columns (total 82 columns):
Province/State    319 non-null object
Country/Region    461 non-null object
Lat               461 non-null float64
Long              461 non-null float64
1/1/20            461 non-null int64
1/2/20            461 non-null int64
1/3/20            461 non-null int64
1/4/20            461 non-null int64
1/5/20            461 non-null int64
1/6/20            461 non-null int64
1/7/20            461 non-null int64
1/8/20            461 non-null int64
1/9/20            461 non-null int64
1/10/20           461 non-null int64
1/11/20           461 non-null int64
1/12/20           461 non-null int64
1/13/20           461 non-null int64
1/14/20           461 non-null int64
1/15/20           461 non-null int64
1/16/20           461 non-null int64
1/17/20           461 non-null int64
1/18/20           461 non-null int64
1/19/20           461 non-null int64
1/20/20           461 non-null int64
1

In [11]:
# For description of the different units being captured from the API,
# please see https://darksky.net/dev/docs
# Requests are being made in SI units

# dataframe to record median maximum temperature
df_tMax = df_template.copy()

# dataframe to record median minimum temperature
df_tMin = df_template.copy()

# dataframe to record median humidity
df_humidity = df_template.copy()

# dataframe to record median UV index
df_uvIndex = df_template.copy()

# dataframe to record median cloud cover
df_cloud = df_template.copy()

# dataframe to record weather summaries
#df_summaries = df_template.copy()

# dataframe to record precipitation probability
df_precipprob = df_template.copy()

# dataframe to record dewpoint
df_dewpoint = df_template.copy()

# dataframe to record pressure
df_pressure = df_template.copy()

# dataframe to record windspeed
df_windspeed = df_template.copy()

# dataframe to record "ozone" reading
df_ozone = df_template.copy()

# dataframe to record sunrise time
df_sunrise = df_template.copy()

# dataframe to record sunset time
df_sunset = df_template.copy()

In [30]:
#Darksky API key
API_KEY = '5f1fd51486fedd234dbaad93d965df91'

#number of seconds in one day
one_day = 86400

t = '1577836800'

latitude = '71.7069'
longitude = '-42.6043'

# sample get request 
#GET https://api.darksky.net/forecast/0123456789abcdef9876543210fedcba/
#42.3601,-71.0589,255657600?exclude=currently,flags

url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t

url = url + '?exclude=currently,flags&units=si'
url

'https://api.darksky.net/forecast/5f1fd51486fedd234dbaad93d965df91/71.7069,-42.6043,1577836800?exclude=currently,flags&units=si'

In [31]:
response = requests.get(url)

In [28]:
response.content

b'{"latitude":6.4281,"longitude":-9.4295,"timezone":"Africa/Monrovia","hourly":{"summary":"Partly cloudy throughout the day.","icon":"partly-cloudy-day","data":[{"time":1577836800,"summary":"Clear","icon":"clear-night","precipIntensity":0,"precipProbability":0,"temperature":22.92,"apparentTemperature":23.72,"dewPoint":21.94,"humidity":0.94,"pressure":1011.7,"windSpeed":1.17,"windGust":1.17,"windBearing":191,"cloudCover":0.05,"uvIndex":0,"visibility":16.093,"ozone":243.9},{"time":1577840400,"summary":"Clear","icon":"clear-night","precipIntensity":0,"precipProbability":0,"temperature":21.47,"apparentTemperature":22.21,"dewPoint":20.99,"humidity":0.97,"pressure":1011.6,"windSpeed":1.04,"windGust":1.04,"windBearing":113,"cloudCover":0.15,"uvIndex":0,"visibility":16.093,"ozone":246.1},{"time":1577844000,"summary":"Clear","icon":"clear-night","precipIntensity":0,"precipProbability":0,"temperature":20.95,"apparentTemperature":21.81,"dewPoint":20.95,"humidity":1,"pressure":1011.3,"windSpeed":0

In [32]:
info = json.loads(response.content)

info['daily']['data'][0]

{'time': 1577761200,
 'summary': 'Foggy throughout the day.',
 'icon': 'cloudy',
 'moonPhase': 0.18,
 'precipIntensity': 0.0273,
 'precipIntensityMax': 0.0629,
 'precipIntensityMaxTime': 1577789100,
 'precipProbability': 0.17,
 'precipType': 'snow',
 'precipAccumulation': 6,
 'temperatureHigh': -30.32,
 'temperatureHighTime': 1577791620,
 'temperatureLow': -47.59,
 'temperatureLowTime': 1577860260,
 'apparentTemperatureHigh': -44.57,
 'apparentTemperatureHighTime': 1577791020,
 'apparentTemperatureLow': -61.73,
 'apparentTemperatureLowTime': 1577860080,
 'dewPoint': -41.89,
 'humidity': 0.48,
 'pressure': 1008.7,
 'windSpeed': 5.37,
 'windGust': 9.02,
 'windGustTime': 1577790180,
 'windBearing': 195,
 'cloudCover': 0.94,
 'uvIndex': 0,
 'uvIndexTime': 1577761200,
 'visibility': 1.055,
 'ozone': 314,
 'temperatureMin': -43.22,
 'temperatureMinTime': 1577840820,
 'temperatureMax': -30.32,
 'temperatureMaxTime': 1577791620,
 'apparentTemperatureMin': -57.55,
 'apparentTemperatureMinTime':

In [14]:
df_tMax.head()

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# pull data from darksky weather API

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = '5f1fd51486fedd234dbaad93d965df91'

# counter
counter = 0


# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            response = requests.get(url)
            counter = counter + 1
            print(counter)
            
            # Putting the API response into the JSON thing
            info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = easy_info['temperatureHigh']
            tMin = easy_info['temperatureLow']
            hum = easy_info['humidity'] * 100
            uvee = easy_info['uvIndex']
            clouds = easy_info['cloudCover'] * 100
            precip = easy_info['precipProbability'] * 100
            dew = easy_info['dewPoint']
            pressure = easy_info['pressure']
            wind = easy_info['windSpeed']
            ozone = easy_info['ozone']
            sunrise = easy_info['sunriseTime']
            sunset = easy_info['sunsetTime']
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset

          
            # API has a daily limit, so we're just taking a break here
            #if (y == 10):
             #   break
                
   # if (counter == 433):
    #    break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyError: 'daily'

In [12]:
df_tMax.head(12)

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,British Columbia,Canada,49.2827,-123.1207,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Victoria,Australia,-37.8136,144.9631,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Queensland,Australia,-28.0167,153.4000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,NaN,Cambodia,11.5500,104.9167,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# test run

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = 'API KEY'

# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            #url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            #url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            #response = requests.get(url)
                
            # Putting the API response into the JSON thing
            #info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            #easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = y + y
            tMin = y + y
            hum = y + y
            uvee = y + y
            clouds = y + y
            precip = y + y
            dew = y + y
            pressure = y + y
            wind = y + y
            ozone = y + y
            sunrise = y + y
            sunset = y + y
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset
            
            # API has a daily limit, so we're just taking a break here
            #if (y == 10):
             #   break
                
    #if (x == '1/22/20'):
     #   break


In [13]:
df_sunset.tail(12)

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
448,NaN,Liberia,6.4281,-9.4295,896,896,896,896,896,896,...,896,896,896,896,896,896,896,896,896,896
449,Curacao,Netherlands,12.1696,-68.9900,898,898,898,898,898,898,...,898,898,898,898,898,898,898,898,898,898
450,NaN,Republic of the Congo,-1.4400,15.5560,900,900,900,900,900,900,...,900,900,900,900,900,900,900,900,900,900
451,NaN,Somalia,5.1521,46.1996,902,902,902,902,902,902,...,902,902,902,902,902,902,902,902,902,902
452,NaN,Tanzania,-6.3690,34.8888,904,904,904,904,904,904,...,904,904,904,904,904,904,904,904,904,904
453,NaN,The Bahamas,24.2500,-76.0000,906,906,906,906,906,906,...,906,906,906,906,906,906,906,906,906,906
454,Virgin Islands,US,18.3358,-64.8963,908,908,908,908,908,908,...,908,908,908,908,908,908,908,908,908,908
455,Cayman Islands,United Kingdom,19.3133,-81.2546,910,910,910,910,910,910,...,910,910,910,910,910,910,910,910,910,910
456,Reunion,France,-21.1351,55.2471,912,912,912,912,912,912,...,912,912,912,912,912,912,912,912,912,912
457,NaN,Barbados,13.1939,-59.5432,914,914,914,914,914,914,...,914,914,914,914,914,914,914,914,914,914


In [53]:
# The real API run

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = '5f1fd51486fedd234dbaad93d965df91'

# Setting a counter for API calls as an emergency break, in case the thing gets out of control
# counter = 0

# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            response = requests.get(url)
            # Incrementing the counter
            # counter = counter + 1
                
            # Putting the API response into the JSON thing
            info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = easy_info['temperatureHigh']
            tMin = easy_info['temperatureLow']
            hum = easy_info['humidity'] * 100
            uvee = easy_info['uvIndex']
            clouds = easy_info['cloudCover'] * 100
            precip = easy_info['precipProbability'] * 100
            dew = easy_info['dewPoint']
            pressure = easy_info['pressure']
            wind = easy_info['windSpeed']
            ozone = easy_info['ozone']
            sunrise = easy_info['sunriseTime']
            sunset = easy_info['sunsetTime']
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y, x] = sunrise
            df_sunset.at[y, x] = sunset
            
 
     

KeyError: 'daily'

In [16]:
df_sunrise.head()

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,NaN,Thailand,15.0000,101.0000,1577835780,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,1577829600,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,NaN,Singapore,1.2833,103.8333,1577833680,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3,NaN,Nepal,28.1667,84.2500,1577841360,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
4,NaN,Malaysia,2.5000,112.5000,1577831700,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [17]:
df_tMax.head(12)

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,NaN,Thailand,15.0000,101.0000,33,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Japan,36.0000,138.0000,6,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,NaN,Singapore,1.2833,103.8333,31,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3,NaN,Nepal,28.1667,84.2500,21,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
4,NaN,Malaysia,2.5000,112.5000,28,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
5,British Columbia,Canada,49.2827,-123.1207,8,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
6,New South Wales,Australia,-33.8688,151.2093,24,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
7,Victoria,Australia,-37.8136,144.9631,25,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
8,Queensland,Australia,-28.0167,153.4000,30,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
9,NaN,Cambodia,11.5500,104.9167,34,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18


In [13]:
df_tMin.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_humidity.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,80,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,79,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,60,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,82,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,88,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,70,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_uvIndex.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_cloud.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,74,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,67,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,48,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_precipprob.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,25,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_dewpoint.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_pressure.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,1026,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1021,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1023,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1015,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_windspeed.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_sunrise.head(12)

,Province/State,Country/Region,Lat,Long,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,...,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,NaN,Thailand,15.0000,101.0000,1577835780,1577922180,1578008580,1578095040,1578181440,1578267840,...,1583191980,1583278320,1583364720,1583451060,1583537400,1583623800,1583710140,1583796480,1583882880,1583969220
1,NaN,Japan,36.0000,138.0000,1577829600,1577916000,1578002400,1578088800,1578175200,1578261600,...,1583183820,1583270100,1583356440,1583442780,1583529060,1583615400,1583701680,1583788020,1583874360,1583960640
2,NaN,Singapore,1.2833,103.8333,1577833680,1577920080,1578006540,1578092940,1578179340,1578265800,...,1583190900,1583277300,1583363700,1583450040,1583536440,1583622840,1583709240,1583795580,1583881980,1583968380
3,NaN,Nepal,28.1667,84.2500,1577841360,1577927760,1578014160,1578100620,1578187020,1578273420,...,1583196420,1583282760,1583369100,1583455440,1583541720,1583628060,1583714400,1583800740,1583887080,1583973420
4,NaN,Malaysia,2.5000,112.5000,1577831700,1577918100,1578004560,1578090960,1578177420,1578263820,...,1583188860,1583275260,1583361660,1583448000,1583534400,1583620800,1583707140,1583793540,1583879940,1583966280
5,British Columbia,Canada,49.2827,-123.1207,1577808540,1577894940,1577981340,1578067740,1578154140,1578240480,...,1583160780,1583247000,1583333280,1583419560,1583505840,1583592120,1583678400,1583764680,1583850960,1583937240
6,New South Wales,Australia,-33.8688,151.2093,1577818140,1577904540,1577991000,1578077460,1578163920,1578250320,...,1583178360,1583264820,1583351280,1583437740,1583524200,1583610600,1583697060,1583783520,1583869980,1583956440
7,Victoria,Australia,-37.8136,144.9631,1577818920,1577905380,1577991840,1578078300,1578164760,1578251220,...,1583179680,1583266140,1583352600,1583439060,1583525520,1583611980,1583698440,1583784900,1583871360,1583957820
8,Queensland,Australia,-28.0167,153.4000,1577818440,1577904900,1577991360,1578077760,1578164220,1578250680,...,1583178120,1583264520,1583350980,1583437380,1583523840,1583610300,1583696700,1583783160,1583869560,1583956020
9,NaN,Cambodia,11.5500,104.9167,1577834460,1577920860,1578007320,1578093720,1578180120,1578266580,...,1583190960,1583277300,1583363700,1583450040,1583536380,1583622780,1583709120,1583795520,1583881860,1583968260


In [13]:
# writing CSVs of what I've got so far

df_tMax.to_csv('./csv/tMax.csv', index=False)

df_tMin.to_csv('./csv/tMin.csv', index=False)

df_humidity.to_csv('./csv/humidity.csv', index=False)

df_uvIndex.to_csv('./csv/uv.csv', index=False)

df_cloud.to_csv('./csv/cloud.csv', index=False)

df_precipprob.to_csv('./csv/precip.csv', index=False)

df_dewpoint.to_csv('./csv/dew.csv', index=False)

df_pressure.to_csv('./csv/pressure.csv', index=False)

df_windspeed.to_csv('./csv/wind.csv', index=False)

df_ozone.to_csv('./csv/ozone.csv', index=False)

df_sunrise.to_csv('./csv/sunrise.csv', index=False)

df_sunset.to_csv('./csv/sunset.csv', index=False)
